Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [39]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [40]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [41]:
ls = np.array([0, 1, 0, 6])

# hogy is van ez az alábbi sor? :)
nls = (np.arange(10) == ls[:,None]).astype(np.float32)
# np.arange -- sorvektor
# ls[:,None] -- oszlopvektor lesz (None jelentése itt: np.newaxis!)
# mindkettőt broadcastoljuk -> lesz 2 db len(labels) x 10-es mátrixunk :)
# ahol a kettő egyenlő, ott True/False (helyett 1/0!) lesz. Menő! :)

print(np.arange(10))
print(ls[:,None])
print(nls)


[0 1 2 3 4 5 6 7 8 9]
[[0]
 [1]
 [0]
 [6]]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]]


In [42]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # a -1 által kitalálja, hogy mennyi legyen ott -- ugye ahány kép van

  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [43]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are attached to the graph.
  # az egész adathalmaz 1 db változó lesz! :)
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    
    tf.truncated_normal([image_size * image_size, num_labels])) # így 82.5% és 82.7% lett
    
    #tf.zeros([image_size * image_size, num_labels])) # így 88.6% és 88.6% lett!!! XXX XXX XXX (ez -35%!)

  biases = tf.Variable(tf.zeros([num_labels]))
  # az egész W az 1 db változó és a b is :)
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.

  # szinte pont, ahogy volt a diákon (csak már az első lépésben a teljes adathalmazon dolgozunk):
  # XW+b for every X_i in input set
  # (azaz itt a biases-nél van egy broadcasting, minden sorhoz hozzáadja!):
  logits = tf.matmul(tf_train_dataset, weights) + biases

  # 1/N sum_i D(S(fenti),L) ::
  # XXX a reduce_mean() honnan is tudja, hogy mely változók szerint? XXX
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  # 0.5 -- ez hát a learning rate! :) + a minimize-ben adom meg, hogy mi szerint
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  # -- értsd: minden nyilván az adott változó épp aktuális értékéből számolva!
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [44]:
print(1-(1000.-886)/(1000-825))


0.348571428571


In [45]:
def accuracy(predictions, labels, verbose=False):
  if verbose:  
    print(np.argmax(predictions, axis=1))
    print(np.argmax(labels, axis=1))
    print(predictions.shape[0])
  return (100.0 * np.sum(np.argmax(predictions, axis=1) == np.argmax(labels, axis=1)) / predictions.shape[0])

p = np.array([[0.1, 0.5, 0.4],[0.1, 0.3, 0.6],[0.2, 0.1, 0.7],[0.3, 0.3, 0.4]])
l = np.array([[0., 1., 0.],[0., 0., 1.],[1., 0., 0.],[0., 0., 1.]])

print(accuracy(p,l,verbose=True))
# na jó, így már kb. világos!
# sorokban vannak az egy obj-ra vonatkozó adatok
# és a 2dim mátrix sorain belül kell egybeejteni a dolgokat,
# ezért axis=1 (ti. az axis-okat a legnagyobb egységtől számoljuk: 0. hányadik sor -- 1. hányadik elem benne)

[1 2 2 2]
[1 2 0 2]
4
75.0


In [46]:
num_steps = 801

with tf.Session(graph=graph) as session:

  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized\n')

  for step in range(num_steps):

    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    #
    #_, l, predictions = session.run([optimizer, loss, train_prediction])
    #
    # az alábbi jobb, mint a fenti,
    # mert a loss-t meg a train_prediction-t elég kiszámolni akkor eval()-lal
    # mikor épp ki akarjuk írni -- és persze így sokkal gyorsabb is!
    session.run(optimizer)

    if (step % 100 == 0):
      print('Loss at step %3d    = %f' % (step, loss.eval()))
      print('Training accuracy   = %.1f%%' % accuracy(
        train_prediction.eval(), train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy = %.1f%%\n' % accuracy(
        valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized

Loss at step   0    = 13.446451
Training accuracy   = 15.0%
Validation accuracy = 14.9%

Loss at step 100    = 2.265620
Training accuracy   = 72.3%
Validation accuracy = 71.7%

Loss at step 200    = 1.846820
Training accuracy   = 75.0%
Validation accuracy = 73.7%

Loss at step 300    = 1.612310
Training accuracy   = 76.4%
Validation accuracy = 74.5%

Loss at step 400    = 1.452225
Training accuracy   = 77.2%
Validation accuracy = 75.0%

Loss at step 500    = 1.331900
Training accuracy   = 77.8%
Validation accuracy = 75.4%

Loss at step 600    = 1.235922
Training accuracy   = 78.3%
Validation accuracy = 75.7%

Loss at step 700    = 1.156865
Training accuracy   = 78.9%
Validation accuracy = 75.8%

Loss at step 800    = 1.090358
Training accuracy   = 79.4%
Validation accuracy = 76.0%

Test accuracy: 82.5%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [47]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    
    tf.truncated_normal([image_size * image_size, num_labels])) # így: 86.4%

    #tf.zeros([image_size * image_size, num_labels])) # így: 89.3% (-21.3% hiba!)
    
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [48]:
print(1-(1000.-893)/(1000-864))

0.213235294118


Let's run it:

In [49]:
num_steps = 3001

with tf.Session(graph=graph) as session:

  tf.initialize_all_variables().run()
  print("Initialized\n")

  for step in range(num_steps):

    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

    # _, l, predictions = session.run(
    #   [optimizer, loss, train_prediction], feed_dict=feed_dict)

    # -- átalakítás, mint fent

    session.run(optimizer, feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %3d = %f" % (step, loss.eval(feed_dict=feed_dict)))
      print("Minibatch accuracy         = %.1f%%" % accuracy(train_prediction.eval(feed_dict=feed_dict), batch_labels))
      print("Validation accuracy        = %.1f%%\n" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step   0 = 10.427471
Minibatch accuracy         = 16.4%
Validation accuracy        = 14.8%

Minibatch loss at step 500 = 1.473832
Minibatch accuracy         = 76.6%
Validation accuracy        = 75.8%

Minibatch loss at step 1000 = 1.403615
Minibatch accuracy         = 75.0%
Validation accuracy        = 76.1%

Minibatch loss at step 1500 = 1.180760
Minibatch accuracy         = 78.9%
Validation accuracy        = 77.9%

Minibatch loss at step 2000 = 1.219984
Minibatch accuracy         = 77.3%
Validation accuracy        = 78.0%

Minibatch loss at step 2500 = 0.670039
Minibatch accuracy         = 84.4%
Validation accuracy        = 78.6%

Minibatch loss at step 3000 = 1.069927
Minibatch accuracy         = 79.7%
Validation accuracy        = 79.2%

Test accuracy: 85.9%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

Fel kell írni "képlettel" a diákon látható ReLU-s ábrát.
A mátrix méretekkel, dimenziókkal elleszek itt sztem. :)

In [50]:
batch_size = 128
hidden_level_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_level_size])) # 88,1%
    #tf.zeros([image_size * image_size, hidden_level_size])) # hú, ez így csak 10% - miért? XXX
    
  # itt lehet kísérletezni azzal, hogy a zeros tensor-ban 1-2 elemet kicserélünk
  # (akkor fent w1 helyett w1_pre kell)
  # 1 db elem 0 helyett 1 => accuracy = 25%!
  # 5 db elem 0 helyett 1 => accuracy = 86%!
  #indices = [[1, 1], [5,5], [15,2], [25,80], [74,31]] # ezen a ponton...
  #values = [1., 1., 1., 1., 1.] # ...ez legyen az értéke
  #shape = [image_size * image_size, hidden_level_size] # ua
  #delta = tf.SparseTensor(indices, values, shape)
  #w1 = w1_pre + tf.sparse_tensor_to_dense(delta)
    
  b1 = tf.Variable(tf.zeros([hidden_level_size]))

  w2 = tf.Variable(
    tf.truncated_normal([hidden_level_size, num_labels]))
    #tf.zeros([hidden_level_size, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # XXX ennyi a tippem: matmul, relu, matmul :)
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), w2) + b2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [51]:
num_steps = 3001

with tf.Session(graph=graph) as session:

  tf.initialize_all_variables().run()
  print("Initialized\n")

  for step in range(num_steps):

    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    # _, l, predictions = session.run(
    #   [optimizer, loss, train_prediction], feed_dict=feed_dict)

    # -- átalakítás, mint fent
    
    session.run(optimizer, feed_dict=feed_dict)
        
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, loss.eval(feed_dict=feed_dict)))
      print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction.eval(feed_dict=feed_dict), batch_labels))
      print("Validation accuracy: %.1f%%\n" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 614.096680
Minibatch accuracy: 57.8%
Validation accuracy: 36.9%

Minibatch loss at step 500: 1.613351
Minibatch accuracy: 92.2%
Validation accuracy: 81.5%

Minibatch loss at step 1000: 1.826458
Minibatch accuracy: 89.8%
Validation accuracy: 80.5%

Minibatch loss at step 1500: 3.155628
Minibatch accuracy: 89.8%
Validation accuracy: 80.6%

Minibatch loss at step 2000: 0.441695
Minibatch accuracy: 95.3%
Validation accuracy: 82.1%

Minibatch loss at step 2500: 0.889209
Minibatch accuracy: 93.0%
Validation accuracy: 82.2%

Minibatch loss at step 3000: 0.655658
Minibatch accuracy: 88.3%
Validation accuracy: 82.1%

Test accuracy: 88.9%
